In [ ]:
#| default_exp create_thumbnails

# Normalize images and create thumbnails

In [ ]:
#| export

import numpy as np
import daft
from PIL import Image

## 1. Normalize

In [ ]:
# | export

def _autocontrast_normed(img_arr: np.array) -> np.array:
    """autocontrast from keras - added normalization"""
    x = img_arr.astype(float)
    mean_before = x.mean()
    ## autocontrast from Keras
    x = x - np.min(x)
    x_max = np.max(x)
    if x_max != 0:  x /= x_max
    x *= 255

    ## return to average lightness of input image
    mean_shift = x.mean() - mean_before

    print(f"Shape after: {x.shape}")
    return np.clip(x - mean_shift, 0, 255).astype("uint8")

In [ ]:
# | export

@daft.udf(return_dtype=daft.DataType.image(mode="RGB"))
def _autocontrast_img(arrs: daft.Series) -> np.array:
    """apply autocontrast to an image column"""
    return [_autocontrast_normed(img) for img in arrs.to_pylist()]

In [ ]:
#| export

def create_autocontrast_col(df: daft.DataFrame, image_col: str="img") -> daft.DataFrame:
    """create autocontrast image column"""
    return df.with_column("img_autocontrast", _autocontrast_img(df["img"]))

In [ ]:
#| hide
 
@daft.udf(return_dtype=daft.DataType.string())
def _get_image_size_type(imcol: daft.Series):
    """return string of dict of daft image size and type"""
    ims = [Image.fromarray(a) for a in imcol.to_pylist()]
    return [f"Shape:{i.size}, format: {i.format}" for i in ims]

## 2. Create thumbnails

### Notes on thumbnails

- default `cell_size` in [PixPlot is 32](https://github.com/pleonard212/pix-plot/blob/84afbd098f24c5a3ec41219fa849d3eb7b3dc57f/pixplot/pixplot.py#L126). This controls atlas creation.
    - actual shape is set by maximum height being cell_size
- default `lod_cell_height` in [PixPlot is 128](https://github.com/pleonard212/pix-plot/blob/84afbd098f24c5a3ec41219fa849d3eb7b3dc57f/pixplot/pixplot.py#L127). This controls thumbs directory.
    - actual shape is set such that maximum side length is lod_cell_height

In [ ]:
#| hide

import nbdev; nbdev.nbdev_export()